In [21]:
import cv2
from cv2 import *
from tkinter import *
from PIL import ImageTk, Image
from time import sleep

In [22]:
cam = VideoCapture(0)
if not(cam.isOpened()):
    print('Error opening camera')

In [23]:
s, img = cam.read()
type(img)

numpy.ndarray

In [24]:
def rotate_img(img, angle):
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    return cv2.warpAffine(img, M, (cols,rows))

In [25]:
def display_img(img, window_name):
    namedWindow(window_name, WINDOW_AUTOSIZE)
    imshow(window_name, img)
    waitKey(0)
    destroyWindow(window_name)

In [26]:
rotated_img = rotate_img(img, 180)

In [10]:
if s:
    display_img(rotated_img, "Microwave")

In [17]:
capture_delay = 0.5 # seconds
window_name = "Microwave Capture"
namedWindow(window_name, WINDOW_AUTOSIZE)
for i in range(120):
    s, img = cam.read()
    if s:
        rotated_img = rotate_img(img, 180)
        imshow(window_name, rotated_img)
        waitKey(1)
        sleep(capture_delay)
destroyWindow(window_name)    

## Create GUI

In [44]:
def update_img():
    s, img = cam.read()
    if s:
        rotated_img = rotate_img(img, 180)
        camera = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)
    camera = Image.fromarray(camera)
    camera = ImageTk.PhotoImage(camera)
    display_img = Label(image=camera)
    display_img.grid(row=0, columnspan=2)

In [43]:
root = Tk()

# Frames...............................
btnRun = Button(root, text="Run")
btnRun.grid(row=2, column=0, sticky=N+S+E+W)

btnUpdateImage = Button(root, text="Update Image")
btnUpdateImage.grid(row=2, column=1, sticky=N+S+E+W)

btnFacingDoor = Button(root, text="Facing Door")
btnFacingDoor.grid(row=2, column=2, sticky=N+S+E+W)


s, img = cam.read()
rotated_img = rotate_img(img, 180)
camera = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)
camera = Image.fromarray(camera)
camera = ImageTk.PhotoImage(camera)
display_img = Label(image=camera)
display_img.grid(row=0, columnspan=2)


root.mainloop()